<a href="https://colab.research.google.com/github/BryceRodgers7/CoTorch/blob/main/DuneBookTopics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1 DOWNLOAD DATA
# begin by downloading the first 3 dune books .txt
import json
import pandas as pd
import requests

# text will usually be ~1 sentence per line, some blank lines, no page numbers or chapter titles
url = 'https://raw.githubusercontent.com/ganesh-k13/shell/master/test_search/www.glozman.com/TextPages/Frank%20Herbert%20-%20Dune.txt'
response = requests.get(url)
lines = []
if response.status_code == 200:
    # Split the text into lines and remove any leading or trailing whitespace
    lines = response.text.splitlines()
    print(f'downloaded {len(lines)} lines')
else:
    print('Failed to download file')

print('\n\nFinished STEP 1: Download Data')

downloaded 8608 lines


Finished STEP 1: Download Data


In [ ]:
# STEP 2 CONSTRUCT DOCUMENTS
# Several options on what to consider a document for training & testing purposes.. all 3 books together as well as each book on its own. 7100 unique words.
#   First book's TF matrix is either 21 chapters, or ~3000 lines/sentences

# re-combine the lines into chapter-sized documents
def combine_lines_into_chapters(lines, chapter_indexes):
    # note this has a hard-coded starting-line of '1' for the first iteration
    # so multiple corpuses need to remove the 0th document to be distinct
    chapters = []
    start = 1
    for index in chapter_indexes:
        chapter = " ".join(lines[start-1:index-1])
        chapters.append(chapter)
        start = index
    return chapters

def print_chapters(chapters):
  for i, chapter in enumerate(chapters):
      print(f"Chapter {i}:")
      print(chapter)
      print()


# chapter start lines (manually derived):
chp_start_idx_bk1 = [11, 184, 310, 422, 571, 635, 729, 888, 1010, 1065, 1201, 1278, 1514, 1601, 1651, 2031, 2604, 2780, 2832, 3043, 3252]
chp_start_idx_bk2 = [3260, 3334, 3539, 3762, 3976, 4226, 4351, 4510, 4606, 4784, 4907, 5101, 5318, 5616, 5765, 6010 ]
chp_start_idx_bk3 = [6018, 6162, 6279, 6460, 6621, 6743, 7021, 7236, 7396, 7552, 7689, 8055 ]
chp_start_idx_all_bks = chp_start_idx_bk1.copy()
chp_start_idx_all_bks.extend(chp_start_idx_bk2)
chp_start_idx_all_bks.extend(chp_start_idx_bk3)
print(f'chp_start_idx_all_bks is {chp_start_idx_all_bks}')
print(f'chp_start_idx_all_bks length is {len(chp_start_idx_all_bks)}')

# appendix start lines
ap1_start_idx_bka = [8060]
ap2_start_idx_bka = [8136]
ap3_start_idx_bka = [8241]
ap4_start_idx_bka = [8263]
trm_start_idx_bk1 = [8282]


# create corpus for first book, combine all lines between idx into documents
dune_bk1_docs = combine_lines_into_chapters(lines, chp_start_idx_bk1)
dune_bk2_docs = combine_lines_into_chapters(lines, chp_start_idx_bk2)
dune_bk3_docs = combine_lines_into_chapters(lines, chp_start_idx_bk3)

# create corpus for initial dune trilogy
dune_all_bks_docs = combine_lines_into_chapters(lines, chp_start_idx_all_bks)

# Remove title page docs
print(f'removing title-page docs...')
# print(f'{dune_bk1_docs[0]}')
# print(f'{dune_bk2_docs[0]}')
# print(f'{dune_bk3_docs[0]}')
# print(f'37 is {chp_start_idx_all_bks[37]} , {dune_all_bks_docs[37]}')
# print(f'21 is {chp_start_idx_all_bks[21]} ,{dune_all_bks_docs[21]}')
# print(f'0 is {dune_all_bks_docs[0]}')
dune_bk1_docs = dune_bk1_docs[1:]
dune_bk2_docs = dune_bk2_docs[1:]
dune_bk3_docs = dune_bk3_docs[1:]
del dune_all_bks_docs[37]
del dune_all_bks_docs[21]
del dune_all_bks_docs[0]


# print each chapter/document on its own line
# print_chapters(dune_bk1_docs)
# print_chapters(dune_bk2_docs)
# print_chapters(dune_bk3_docs)
# print_chapters(dune_all_bks_docs)
print('\n\nFinished STEP 2: Constructing Documents')

chp_start_idx_all_bks is [11, 184, 310, 422, 571, 635, 729, 888, 1010, 1065, 1201, 1278, 1514, 1601, 1651, 2031, 2604, 2780, 2832, 3043, 3252, 3260, 3334, 3539, 3762, 3976, 4226, 4351, 4510, 4606, 4784, 4907, 5101, 5318, 5616, 5765, 6010, 6018, 6162, 6279, 6460, 6621, 6743, 7021, 7236, 7396, 7552, 7689, 8055]
chp_start_idx_all_bks length is 49
removing title-page docs...


Finished STEP 2: Constructing Documents


In [ ]:
# STEP 3 SANITIZE DATA
# lowercase, remove punctuation, remove stop-words, remove non-alphanumeric, stem words
# TODO: LEMMATIZATION

import re
import string
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize the Porter stemmer and get the list of English stop words
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
fix_words = {}
pre_fix_words = {'--':' ', }


# fix_words = {'abruptli':'abrupt', 'accident':'accid', 'accusingli':'accus',
#              'administr':'administ', 'aliaoftheknif':'alia', 'amplifi':'amplif',
#              'ancestri':'ancestr', 'assumpt':'assum', 'caladanin':'caladan',
#              'cinnamonheavi':'cinnamon', 'wormsign':'worm'}

def pre_fix_words(document):
  # do a regex replace

  return document

def remove_punctuations(document):
  punctuations = string.punctuation
  return document.translate(str.maketrans('', '', punctuations))

def remove_stopwords(document):
  return " ".join([word for word in document.split() if word not in stop_words])

def remove_special_characters(document):
  document = re.sub('[^a-zA-Z0-9]', ' ', document)
  # other regex subs:
  # limit spaces to 1
  document = re.sub('\s+', ' ', document)
  # replace double-dash with space
  document = re.sub('--', ' ', document)
  return document

def stem_words(document):
  return " ".join([stemmer.stem(word) for word in document.split()])

def lemmatize_words(document):
  return " ".join([lemmatizer.lemmatize(word) for word in document.split()])

def fix_words_nuke(document):
  result = ""
  for word in document.split():
    if (word in fix_words):
      result+=f' {fix_words[word]}'
      fixed_words+=1
    else:
      result+=f' {word}'
  return result

def preprocess_document(document):
    fixed_words = 0
    document = document.lower()
    document = remove_punctuations(document)
    document = remove_stopwords(document)
    document = remove_special_characters(document)
    document = stem_words(document)
    document = lemmatize_words(document)
    processed_document = fix_words_nuke(document)
    print(f' .. done! Fixed {fixed_words} words in this document')

    return processed_document

def preprocess_corpus(corpus, name):
    preprocessed_corpus = []
    for i, document in enumerate(corpus):
        print(f'preprocessing corpus {name} document {i+1}..', end = '')
        preprocessed_document = preprocess_document(document)
        preprocessed_corpus.append(preprocessed_document)
    print(f'finished preprocessing {name}!\n\n')
    return preprocessed_corpus


# Preprocess Dune Book 1 corpus
dune_bk1_preprocessed = preprocess_corpus(dune_bk1_docs, 'dune book 1')

# Preprocess Dune Book 1 corpus
dune_bk2_preprocessed = preprocess_corpus(dune_bk2_docs, 'dune book 2')

# Preprocess Dune Book 1 corpus
dune_bk3_preprocessed = preprocess_corpus(dune_bk3_docs, 'dune book 3')

# Preprocess Dune Trilogy corpus
dune_all_bks_preprocessed = preprocess_corpus(dune_all_bks_docs, 'dune trilogy')

# Print dune book 1 preprocessed corpus
for i, document in enumerate(dune_bk1_preprocessed):
    print(f"Document {i+1}: {document}")

print('\n\nFinished STEP 3: Sanitize Data')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


preprocessing corpus dune book 1 document 1.. .. done! Fixed 0 words in this document
preprocessing corpus dune book 1 document 2.. .. done! Fixed 0 words in this document
preprocessing corpus dune book 1 document 3.. .. done! Fixed 0 words in this document
preprocessing corpus dune book 1 document 4.. .. done! Fixed 0 words in this document
preprocessing corpus dune book 1 document 5.. .. done! Fixed 0 words in this document
preprocessing corpus dune book 1 document 6.. .. done! Fixed 0 words in this document
preprocessing corpus dune book 1 document 7.. .. done! Fixed 0 words in this document
preprocessing corpus dune book 1 document 8.. .. done! Fixed 0 words in this document
preprocessing corpus dune book 1 document 9.. .. done! Fixed 0 words in this document
preprocessing corpus dune book 1 document 10.. .. done! Fixed 0 words in this document
preprocessing corpus dune book 1 document 11.. .. done! Fixed 0 words in this document
preprocessing corpus dune book 1 document 12.. .. do

In [ ]:
# STEP 4: CREATE TDF MATRIX
import numpy as np

# Step 1: Create a vocabulary list
vocabulary = sorted(set(word for doc in dune_bk1_preprocessed for word in doc.split()))

# Step 2: Create an empty matrix
dtf_matrix = np.zeros((len(vocabulary), len(dune_bk1_preprocessed)), dtype=int)

# Step 3: Fill in the matrix
for doc_idx, doc in enumerate(dune_bk1_preprocessed):
    for term in doc.split():
        term_idx = vocabulary.index(term)
        dtf_matrix[term_idx, doc_idx] += 1

# Print the DTF matrix
print(f'Document-Term Frequency Matrix is {dtf_matrix.shape[0]} x {dtf_matrix.shape[1]}:')

# highlight possible duplicates, manually verify atomicity of data
print('\n\nhighlighting duplicates:\n')
lastWord = 'abc'
dups = 0
for word in vocabulary:
  if (lastWord in word):
    dups+=1
    print(f'{lastWord} is a subset of {word}')
  lastWord = word

print(f'num possible dups is {dups}')
# alter the matrix to combine like-terms

# print full vocabulary to see terms near the actual duplicates
print('\n\n\n*********FULL VOCABULARY*********\n\n\n')
i = 0
for word in vocabulary:
  if i%8==0:
    print(f'{word} ')
  else:
    print(f'{word} ', end='')
  i+=1



Document-Term Frequency Matrix is 5488 x 20:



highlighting duplicates:

abrupt is a subset of abruptli
accid is a subset of accident
accus is a subset of accusingli
ach is a subset of achiev
act is a subset of action
ad is a subset of adam
add is a subset of addict
administ is a subset of administr
age is a subset of agenc
ago is a subset of agon
agon is a subset of agoni
agre is a subset of agreement
ah is a subset of ahah
ahead is a subset of aheadin
ahh is a subset of ahhh
ahhh is a subset of ahhhh
air is a subset of airborn
aircraft is a subset of aircraftsoft
alarm is a subset of alarmso
alb is a subset of alburhan
algaib is a subset of algaiba
alia is a subset of aliaoftheknif
aliv is a subset of aliveturn
alli is a subset of allianc
alon is a subset of along
along is a subset of alongsid
amaz is a subset of amazementa
amplif is a subset of amplifi
analysi is a subset of analysisin
ancestr is a subset of ancestri
anoth is a subset of anotherthat
arc is a subset of arch
arch is 

In [ ]:
# STEP 4: CREATE TF-IDF MATRIX
from sklearn.feature_extraction.text import TfidfVectorizer

# return both tfidf matrix and the vectorizer
def createTfidfMatrix(corpus):
  # Create TfidfVectorizer
  vectorizer = TfidfVectorizer()

  # Fit the vectorizer to the corpus and transform the corpus into TF-IDF matrix
  tfidf_matrix = vectorizer.fit_transform(corpus)

  return [tfidf_matrix, vectorizer]

# tfidf_vectorizer[0] = matrix, [1] = vectorizer
def printTopTerms(tfidf_vectorizer):
  tfidf_matrix = tfidf_vectorizer[0]
  vectorizer = tfidf_vectorizer[1]
  corpus_top_scores = []

  # print the TF-IDF scores for each row/document
  for doc_idx in range(tfidf_matrix.shape[0]):
    doc_tfidf_scores = tfidf_matrix[doc_idx].toarray()[0]

    # Get feature names (words) from the vectorizer
    feature_names = vectorizer.get_feature_names_out()

    # Create a dictionary to store feature names and their corresponding TF-IDF scores
    feature_tfidf_dict = dict(zip(feature_names, doc_tfidf_scores))

    # Sort the dictionary by TF-IDF scores in descending order
    sorted_features = sorted(feature_tfidf_dict.items(), key=lambda x: x[1], reverse=True)


    top_n = 10

    # Print the top N features with their TF-IDF scores
    # print(f"\n\n*** Top {top_n} TF-IDF scores in (document) chapter {doc_idx+1} ***")

    doc_top_scores = []
    for feature, score in sorted_features[:top_n]:
        # print(f"{feature}: {score}")
        doc_top_scores.append(feature)
    corpus_top_scores.append(doc_top_scores)
  return corpus_top_scores

print('creating dune book 1 tfidf matrix..', end='')
# create TF-IDF matrix for Dune book 1 corpus
# dune_bk1_corpus = dune_bk1_preprocessed.copy()
dune_bk1_tfidf_vectorizer = createTfidfMatrix(dune_bk1_preprocessed)
# print book1 tfidf scores
dune_bk1_top_scores = printTopTerms(dune_bk1_tfidf_vectorizer)
print('complete')

print('creating dune book 2 tfidf matrix..', end='')
# create TF-IDF matrix for Dune book 2 corpus
# dune_bk2_corpus = dune_bk2_preprocessed.copy()
dune_bk2_tfidf_vectorizer = createTfidfMatrix(dune_bk2_preprocessed)
# print book1 tfidf scores
dune_bk2_top_scores = printTopTerms(dune_bk2_tfidf_vectorizer)

print('creating dune book 3 tfidf matrix..', end='')
# create TF-IDF matrix for Dune book 3 corpus
# dune_bk3_corpus = dune_bk3_preprocessed.copy()
dune_bk3_tfidf_vectorizer = createTfidfMatrix(dune_bk3_preprocessed)
# print book1 tfidf scores
dune_bk3_top_scores = printTopTerms(dune_bk3_tfidf_vectorizer)

print('creating dune Trilogy tfidf matrix..', end='')
# create TF-IDF matrix for Dune Trilogy corpus
# dune_all_bks_corpus = dune_all_bks_preprocessed.copy()
dune_all_bks_tfidf_vectorizer = createTfidfMatrix(dune_all_bks_preprocessed)
# print book1 tfidf scores
dune_all_bks_top_scores = printTopTerms(dune_all_bks_tfidf_vectorizer)

print('\n\n\ndune book 1 top scores comparison:')
# compare top scores from bk1 to bk1 in 'all_bks' top scores
for i, scores in enumerate(dune_bk1_top_scores):
  delta = list(set(scores) ^ set(dune_all_bks_top_scores[i]))
  if (len(delta) > 0):
    print(f'\ndocument {i} has delta {delta}')
    print(f'book 1 scores were {scores}')
    print(f'trilogy scores were {dune_all_bks_top_scores[i]}')
  else:
    print(f'\ndocument {i} has same top scores in both corpuses')

offset = len(dune_bk1_top_scores)
print('\n\n\ndune book 2 top scores comparison:')
# compare top scores from bk1 to bk1 in 'all_bks' top scores
for i, scores in enumerate(dune_bk2_top_scores):
  delta = list(set(scores) ^ set(dune_all_bks_top_scores[i+offset]))
  if (len(delta) > 0):
    print(f'\ndocument {i} has delta {delta}')
    print(f'book 2 scores were {scores}')
    print(f'trilogy scores were {dune_all_bks_top_scores[i]}')
  else:
    print(f'\ndocument {i} has same top scores in both corpuses')

offset += len(dune_bk2_top_scores)
print('\n\n\ndune book 3 top scores comparison:')
# compare top scores from bk1 to bk1 in 'all_bks' top scores
for i, scores in enumerate(dune_bk3_top_scores):
  delta = list(set(scores) ^ set(dune_all_bks_top_scores[i+offset]))
  if (len(delta) > 0):
    print(f'\ndocument {i} has delta {delta}')
    print(f'book 3 scores were {scores}')
    print(f'trilogy scores were {dune_all_bks_top_scores[i]}')
  else:
    print(f'\ndocument {i} has same top scores in both corpuses')

print('\n\nFinished STEP 4: Create TF-IDF Matrix\n\n')

'''
BOOK 1
ANALYSIS


I have seen the movie, but I have not read the books.
This is my attempt to link well-known scenes in the movie,
to chapters in book 1, by analyzing all 20 chapters' top TF-IDF scores.


document 0 (chapter 1) is clearly the 'gom jabbar' chapter, the top 10 include:
  'gom' 'jabbar' 'paul' 'old' 'woman' 'box'...
  'gom' and 'jabbar' are not in any other top-10

document 3 is clearly the 'shield-training' chapter, its top 10 include:
  'paul' 'halleck' 'rapier' 'gurney' 'table' 'mood' 'lad'...
  'mood' and 'rapier' are not in any other top-10
  only one other chapter has a weapon (word) in its top-10

document 8 is clearly the hunter-seeker attack chapter, its top 10 words include:
  'paul' 'room' 'headboard' 'hunterseek(er)' 'bed' 'seeker' 'oper(ator)'
  'headboard' and 'hunerseek(er)' and 'seeker' never show in any other top-10.

document 15 is the sabotaged spice-harvester chapter, its top 10 words include:
  'kyne(s)' 'duke' 'halleck' 'crawler' 'worm' 'paul' 'sand'
  this is the first chapter where 'worm' or 'crawler' or 'sand' are in the top 10


Nearly every chapter's top-10 included multiple character names along with 'said'
A good take-away is that - like the movie - the book is filled with dialogue and not 'action'

Although spice is vital to understanding Dune, no chapter or scene is 'about' spice.
'spice' and 'melange' are never in the top-10,
this is because it is not a physical 'subject' in any chapter.
In the movie it appears as minor dialogue,
plus one 'filmbook' scene (the most prominent of the three).
  There are 3 'filmbook' scenes in the movie,
  the shortest being ~10 seconds and the longest ~1 minute.

Thus if we look granularly enough,
we should be able to find a document where 'spice' scores high on TF-IDF!

'''

creating dune book 1 tfidf matrix..complete
creating dune book 2 tfidf matrix..creating dune book 3 tfidf matrix..creating dune Trilogy tfidf matrix..


dune book 1 top scores comparison:

document 0 has delta ['hand', 'pain']
book 1 scores were ['gom', 'jabbar', 'paul', 'mother', 'said', 'jessica', 'hand', 'old', 'woman', 'box']
trilogy scores were ['paul', 'said', 'mother', 'gom', 'jabbar', 'pain', 'old', 'jessica', 'box', 'woman']

document 1 has delta ['duke', 'nephew']
book 1 scores were ['piter', 'baron', 'feydrautha', 'said', 'feyd', 'uncl', 'globe', 'mentat', 'nephew', 'know']
trilogy scores were ['piter', 'baron', 'said', 'feydrautha', 'feyd', 'globe', 'mentat', 'uncl', 'duke', 'know']

document 2 has delta ['tell', 'poem']
book 1 scores were ['jessica', 'girl', 'mother', 'reverend', 'paul', 'old', 'dream', 'woman', 'said', 'tell']
trilogy scores were ['jessica', 'girl', 'paul', 'reverend', 'old', 'mother', 'woman', 'dream', 'said', 'poem']

document 3 has same top scores in b

"\nBOOK 1\nANALYSIS\n\n\nI have seen the movie, but I have not read the books.\nThis is my attempt to link well-known scenes in the movie,\nto chapters in book 1, by analyzing all 20 chapters' top TF-IDF scores.\n\n\ndocument 0 (chapter 1) is clearly the 'gom jabbar' chapter, the top 10 include:\n  'gom' 'jabbar' 'paul' 'old' 'woman' 'box'...\n  'gom' and 'jabbar' are not in any other top-10\n\ndocument 3 is clearly the 'shield-training' chapter, its top 10 include:\n  'paul' 'halleck' 'rapier' 'gurney' 'table' 'mood' 'lad'...\n  'mood' and 'rapier' are not in any other top-10\n  only one other chapter has a weapon (word) in its top-10\n\ndocument 8 is clearly the hunter-seeker attack chapter, its top 10 words include:\n  'paul' 'room' 'headboard' 'hunterseek(er)' 'bed' 'seeker' 'oper(ator)'\n  'headboard' and 'hunerseek(er)' and 'seeker' never show in any other top-10.\n\ndocument 15 is the sabotaged spice-harvester chapter, its top 10 words include:\n  'kyne(s)' 'duke' 'halleck' 'cra

In [ ]:
# STEP 5: PERFORM KMEANS TOPIC ANALYSIS

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(dune_bk1_preprocessed)

# K-means clustering
k = 4  # Number of clusters
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(tfidf_matrix)

# Get the top terms for each cluster
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

for i in range(k):
    print(f"Cluster {i + 1}:")
    top_terms = [terms[ind] for ind in order_centroids[i, :5]]  # Top 5 terms
    print(", ".join(top_terms))

In [ ]:
# not sure if this is necessary
pip install pyLDAvis

SyntaxError: invalid syntax (<ipython-input-1-eebda11a0cf2>, line 2)

In [ ]:
pip install BERTopic

In [ ]:
from bertopic import BERTopic
topic_model = BERTopic(embedding_model='all-MiniLM-L6-v2')
topics, probs = topic_model.fit_transform(dune_bk1_preprocessed)

# topics for corpus with only few dozen documents is not good with BERT
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,22,-1_said_paul_jessica_one,"[said, paul, jessica, one, thought, duke, baro...",[manual muaddib princess irulan whole theori w...


In [ ]:
import unicodedata
import numpy as np
import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')
print(string.punctuation)

ModuleNotFoundError: No module named 'ntlk'

In [ ]:

import pandas as pd
import numpy as np
import csv

import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel

from pprint import pprint

# be sure to split a doc into individual words
tokenized_documents = [doc.split() for doc in dune_bk1_preprocessed]

dictionary = corpora.Dictionary(tokenized_documents)
words_list = list(dictionary.token2id.keys())
words_list.sort()
dune_bk1_TDF = [dictionary.doc2bow(doc) for doc in tokenized_documents]
# Specify the file path
file_path = 'dune_bk1_tdf.csv'

# Write the data to a CSV file
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(dune_bk1_TDF)

print(f"CSV file saved to {file_path}")





CSV file saved to dune_bk1_tdf.csv


In [ ]:
df = pd.DataFrame.from_dict(dune_bk1_TDF)

In [ ]:
df.shape

(22, 7186)

In [ ]:
from IPython.display import HTML
import base64

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df)

In [ ]:
for i, doc in enumerate(dune_bk1_TDF):
  print(f'\ndocument {i}')
  for term_id, freq in doc:
    term = dictionary[term_id]
    print(f'({term_id}={term}, {freq})', end=", ")


# PICKUP HERE

document 0
(1, 1), (1965, 1), (book, 1), (copyright, 1), (dune, 2), (frank, 1), (herbert, 1), document 1
(dune, 4), (57th, 1), (abil, 2), (abomin, 1), (abruptli, 1), (accept, 1), (accord, 1), (ach, 2), (acquir, 1), (across, 1), (administ, 1), (affair, 1), (age, 3), (agon, 1), (agoni, 2), (ahah, 1), (ahead, 1), (allow, 1), (almost, 3), (alon, 3), (along, 1), (alreadi, 1), (alway, 2), (among, 1), (ancestri, 1), (ancient, 3), (anger, 1), (angri, 1), (anim, 8), (anoth, 4), (answer, 2), (aortal, 1), (appar, 1), (appear, 1), (appreci, 1), (apprehens, 1), (approach, 1), (argu, 1), (arm, 2), (around, 4), (arous, 1), (arrakeen, 1), (arraki, 12), (ask, 6), (asleep, 2), (assassin, 1), (associ, 1), (astonish, 1), (atreid, 5), (attent, 4), (auma, 1), (avenu, 2), (avoid, 2), (awak, 3), (awaken, 2), (awar, 4), (away, 9), (awe, 1), (awok, 1), (axe, 1), (axi, 1), (back, 6), (background, 1), (balanc, 1), (baliset, 1), (bashar, 1), (basic, 2), (bath, 1), (beam, 1), (becam, 2), (becom, 1), (bed, 4), (bedr

In [ ]:



row_idx = []
for i, doc in enumerate(tokenized_documents, start=1):
  row_idx.append(f'chapter {i}')

df = pd.DataFrame(index=row_idx, columns=dictionary)

# Iterate over the documents in TDF variable to grab its term-frequencies
for i, doc in enumerate(dune_bk1_TDF, start=1):
    for term_id, freq in doc:
        term = dictionary[term_id]  # Get the term from the dictionary

        # spit out to file here instead
        df.loc[f'chapter {i}', term] = freq
    # Append the empty row to df


# get term totals
overall_term_frequencies = {}
for term in df.columns:
  overall_term_freq = 0
  for doc in df.index:
    overall_term_freq =+ df.loc[doc, term]
  overall_term_frequencies[term] = overall_term_freq

# Sort the terms by frequency
sorted_terms = sorted(overall_term_frequencies.items(), key=lambda x: x[1])

# Print the most frequent terms
for term, freq in sorted_terms[:10]:  # Change 10 to however many terms you want to see
    print(f"Term: {term}, Frequency: {freq}")

# start to fix corpus




# Set number of topics
num_topics = 20
# Build LDA model. Corpus for lda model is TDF
# dune_bk1_lda_model = LdaModel(corpus=dune_bk1_TDF, id2word=dictionary, num_topics=num_topics)
dune_bk1_lda_model = LdaModel(corpus=dune_bk1_TDF, id2word=dictionary, num_topics=num_topics, random_state=42, passes=10, alpha='auto', per_word_topics=True)

# Print the keywords for each topic
pprint(dune_bk1_lda_model.print_topics(num_words=20))

Streaming output truncated to the last 5000 lines.
<ipython-input-15-c8b6ccc4d948>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[f'chapter {i}', term] = freq
<ipython-input-15-c8b6ccc4d948>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[f'chapter {i}', term] = freq
<ipython-input-15-c8b6ccc4d948>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) ins

Term: 0, Frequency: nan
Term: 1, Frequency: nan
Term: 2, Frequency: nan
Term: 3, Frequency: nan
Term: 4, Frequency: nan
Term: 5, Frequency: nan
Term: 6, Frequency: nan
Term: 7, Frequency: nan
Term: 8, Frequency: nan
Term: 9, Frequency: nan
[(0,
  '0.020*"said" + 0.013*"jessica" + 0.012*"duke" + 0.008*"paul" + '
  '0.007*"thought" + 0.006*"yueh" + 0.006*"hawat" + 0.006*"water" + '
  '0.005*"man" + 0.005*"one" + 0.005*"look" + 0.005*"leto" + 0.005*"kyne" + '
  '0.005*"could" + 0.004*"harkonnen" + 0.004*"know" + 0.004*"see" + '
  '0.004*"room" + 0.004*"hand" + 0.004*"ask"'),
 (1,
  '0.025*"said" + 0.016*"duke" + 0.016*"paul" + 0.011*"kyne" + 0.009*"halleck" '
  '+ 0.008*"hawat" + 0.007*"man" + 0.006*"one" + 0.006*"ask" + 0.005*"fremen" '
  '+ 0.005*"look" + 0.005*"thought" + 0.005*"men" + 0.005*"back" + '
  '0.005*"leto" + 0.004*"tabl" + 0.004*"like" + 0.004*"harkonnen" + '
  '0.004*"sire" + 0.004*"shield"'),
 (2,
  '0.002*"paul" + 0.002*"said" + 0.001*"duke" + 0.001*"jessica" + '
  '0.00

In [ ]:
# ATTEMPT TO COMBINE LIKE-TERMS
# Initialize a counter for the frequency
term_frequency = 0
reg_freq = 0
similar_terms = []
# Iterate over the TDF variable
for doc in dune_bk1_TDF:
    for term_id, freq in doc:
        term = dictionary[term_id]  # Get the term from the dictionary
        match = re.match(r'spic.', term)
        if match:
          term_frequency += freq
          if (term not in similar_terms):
            similar_terms.append(term)
        if (term == 'spice'):
          reg_freq += freq
# Print the frequency of the term
print(f"Frequency of spice proper: {reg_freq}")
print(f"Frequency of spice-terms: {term_frequency}")
print(f"List of various spice-terms: {similar_terms}")

In [ ]:
dune_bk1_lda_model.print_topic(1, 10)

'0.045*"first day muaddib rode street arrakeen famili peopl along way recal legend propheci ventur shout mahdi shout question statement yet could hope one foretold lisan algaib voic outer world attent focus mother heard bene gesserit obviou like lisan algaib manual muaddib princess irulan duke found thufir hawat alon corner room guard direct sound men set commun equip adjoin room place fairli quiet duke glanc around hawat aros paperclutt tabl greenwal enclosur addit tabl three suspensor chair harkonnen h hastili remov leav imperfect color patch chair liber quit safe hawat said paul sire left confer room im hope hell get rest without distract hawat nod cross door adjoin room close shut nois static electron spark thufir leto said imperi harkonnen stockpil spice attract attent mlord duke purs lip storehous suscept destruct rais hand hawat start speak ignor emperor hoard hed secretli enjoy harkonnen embarrass baron object someth destroy cannot openli admit hawat shook head weve men spare s

In [ ]:

coherence_model_lda = CoherenceModel(model=dune_bk1_lda_model, texts=tokenized_documents, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.2997668409482797


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvisimport gensim

pyLDAvis.enable_notebook()
vis_data = gensimvis.prepare(lda_model, corpus, werds)

# Display the visualization
pyLDAvis.display(vis_data)